## **Tests a trained model**
### **Author:** Alessandro Ulivi (ale.ulivi@gmail.com)
### **Start day (yyyy/mm/dd):** 2024/10/21
### **Description**
#### The notebook loads a model and tests it on a test set.

### **Requirements**
#### The notebook uploads a saved checkpoint from a trained model. By default the checkpoint is expected in a folder named "checkpoints"
#### The notebook expects a folder named "tests" in the work directory, to save summary writer's outputs.
#### The notebook runs on the pip2_segmentation environment and using the scripts of the pip2_segmentation project. Refer to https://github.com/AlessandroUlivi/pip2_segmentation.

##**THIS NOTEBOOK IS VERY EARLY IN IT'S DEVELOPMENT PHASE**

In [1]:
# load tensorboard extension
%load_ext tensorboard

In [2]:
#Import required modules
import datetime
from functools import partial
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import RandomSampler, DataLoader, Subset
from dataprep.data_preparation import make_dataset, add_channel, to_tensor, compose
from models.unet import UNet
from utils.utils_funct import dict2mdtable, load_checkpoint
from modeltest.test_model import test_model
from metrics.metric import DiceCoefficient, DiceLoss, DiceBCELoss

from torch.utils.tensorboard import SummaryWriter
# import torchvision.transforms.v2 as transforms_v2

In [ ]:
test_input_data_dir = r""
test_label_data_dir = r""

In [4]:
# Indicate trasformations - #NOTE: Data are not normalized as the normalization had been done at the moment of dataset creation and before chunking the images
test_data_transformations = [add_channel, to_tensor]
test_trafos = trafos = partial(compose, transforms=test_data_transformations)

#create the test dataset
test_dataset = make_dataset(test_input_data_dir, test_label_data_dir, transform=test_trafos, shuffle_data=True, stack_axis=0)


In [5]:
# open tensorboard inside of our notebook
%tensorboard --logdir tests

Reusing TensorBoard on port 6007 (pid 23760), started 3:54:51 ago. (Use '!kill 23760' to kill it.)

In [ ]:
#=========
# # pass data to DataLoader
batch_size=1

#only work on a small subset of the data, for the moment
num_test_samples = 4
test_sample_ds = Subset(test_dataset, np.arange(num_test_samples))
test_sample_sampler = RandomSampler(test_sample_ds)
test_loader = DataLoader(test_sample_ds, sampler=test_sample_sampler, batch_size=batch_size)

#=========
# pass to device
# if torch.cuda.is_available:
#     print("using gpu")
#     device = torch.device("cuda")
# else:
#     print("using cpu")
#     device = torch.device("cpu")
device = torch.device("cpu")

#=========
# set model's parameters
final_activation="Sigmoid"
depth = 3
num_fmaps = 64
fmap_inc_factor = 4
downsample_factor = 2
kernel_size = 3
padding = "valid"
upsample_mode = "nearest"
unet_model = UNet(depth=depth,
                  in_channels=1,
                  out_channels=1,
                  final_activation=final_activation,
                  num_fmaps=num_fmaps,
                  fmap_inc_factor=fmap_inc_factor,
                  downsample_factor=downsample_factor,
                  kernel_size=kernel_size,
                  padding=padding,
                  upsample_mode=upsample_mode).to(device)

#=========
# set loss function
# loss_function = nn.BCELoss()
# loss_function = DiceLoss()
loss_function = DiceBCELoss() #for the moment it seems that this is the best
# loss_function = BCE_EdgeDiceLoss()
use_loss_weights_range=True
bce_weight = 1
dice_weight = 1
loss_function_range = [0,2]

#=========
# set optimizer
lr = 1e-4
optimizer = torch.optim.Adam(unet_model.parameters(), lr=lr)

#=========
# set metrics
bin_threshold=0.5
metric = DiceCoefficient()

#=========
# indicate key
# runs_counter = get_var_value(filename="varstore.dat")
my_key  = "test_"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# set logger's parameters
logger = SummaryWriter(f"tests/{my_key}")



In [ ]:
#=========
#=========
#=========
#THIS PART OF THE CODE IS NOT PROPERLY TESTED.
#IN ADDITION, global_step MUST BE CALCULATED TO ENTER avg_loss_val, avg_metric_val IN SUMMARY WRITER
#step can be calculated as n_epochs (training) * len(train_loader) (potentially +len(train_loader) need to verify)
#=========
#=========
#=========





#=========
#load checkpoint of the model to test
checkpoint_save_path = "checkpoints"
checkpoint_key = "20241113-2343746"
model, optimizer, epoch = load_checkpoint(model=unet_model,
                                          path=checkpoint_save_path,
                                          optimizer=optimizer,
                                          key=checkpoint_key)


#=========
# model's training without validation
avg_loss_val, avg_metric_val = test_model(model=model,
                                          loader=test_loader,
                                          loss_function=loss_function,
                                          metric=metric,
                                          bin_threshold=bin_threshold,
                                          use_loss_weights_range=use_loss_weights_range,
                                          bce_weight=bce_weight,
                                          dice_weight=dice_weight,
                                          loss_function_range=loss_function_range,
                                          step=None,
                                          tb_logger=None,
                                          device=None,
                                          x_dim=[-2,-1],
                                          y_dim=[-2,-1])

#log avg_loss_val, avg_metric_val to tensorboard
# TO DO: CALCULATE STEP
global_step = 1000 #this is just a dummy number
logger.add_scalar(tag="test_avg_loss", scalar_value=avg_loss_val, global_step=global_step)
logger.add_scalar(tag="test_avg_metric", scalar_value=avg_metric_val, global_step=global_step)

#=========
#log all hyperparameters as text in Tensorboard

#transform the rescaling range of the loss function into a string
str_loss_function_range = f"[{loss_function_range[0]},{loss_function_range[1]}]"

#form a dictionary to with all hyperparameters to be logged
hparam_dict = {"train_checkpoint_key": checkpoint_key,
               "batch_size":str(batch_size),
                "final_activation":final_activation,
                "depth":str(depth),
                "num_fmaps":str(num_fmaps),
                "fmap_inc_factor":str(fmap_inc_factor),
                "downsample_factor":str(downsample_factor),
                "kernel_size":str(kernel_size),
                "padding":padding,
                "upsample_mode":upsample_mode,
                "loss_function":str(loss_function),
                "use_loss_weights_range":str(use_loss_weights_range),
                "bce_weight":str(bce_weight/(bce_weight+dice_weight)),
                "dice_weight":str(dice_weight/(bce_weight+dice_weight)),
                "loss_function_range":str_loss_function_range,
                "optimizer":str(optimizer),
                "bin_threshold":str(bin_threshold),
                "metric":str(metric)}

#transform the dictionary in a table-like string object
hparam_table_like = dict2mdtable(hparam_dict, key='Name', val='Value', transform_2_string=False)

#log the text in Tensorboard summary of the run
logger.add_text('Hyperparams', hparam_table_like, 1)


c:\Users\aless\OneDrive\Desktop\Ale\personal\projects\pip2_segmentation\code\utils.py:336: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint=torch.load(load_path)


FileNotFoundError: [Errno 2] No such file or directory: 'checkpoints\\20241113-2343746.pt'